In [1]:
import pandas as pd
import cobra
from cobra import Reaction, Metabolite

### Theoretical Yield Calculations with iML1515 model with added oleochemical reactions / metabolites. 

In [2]:
# Define product names and corresponding model exchange reaction IDs as a dictionary
objectives = {'Octanol':'EX_OH_C080_LPAREN_e_RPAREN_', 
              'Decanol': 'EX_OH_C100_LPAREN_e_RPAREN_',
              'Dodecanol': 'EX_OH_C120_LPAREN_e_RPAREN_',
              'Tetradecanol': 'EX_OH_C140_LPAREN_e_RPAREN_',
              '2-Heptanone': 'EX_MK_C080_LPAREN_e_RPAREN_',
              '2-Nonanone': 'EX_MK_C100_LPAREN_e_RPAREN_',
              '2-Undecanone': 'EX_MK_C120_LPAREN_e_RPAREN_',
              '2-Tridecanone': 'EX_MK_C140_LPAREN_e_RPAREN_',
              'Octanoic Acid': 'EX_octa_e',
              'Decanoic Acid': 'EX_dca_e',
              'Dodecanoic Acid': 'EX_ddca_e',
              'Tetradecanoic Acid': 'EX_ttdca_e'}

# Create DataFrames for storing the FBA solutions, and the theoretical yields
iML1515_solution_df = pd.DataFrame(index = objectives.keys(), columns = ['Reverse Beta Oxidation','Thioesterase (ΔfadAI)','PhaG PhaJ ΔfadDEK','PhaG PhaJ tdTER BktB ΔfadBDK'])
iML1515_theoretical_yeilds_df = pd.DataFrame(index = objectives.keys(), columns = ['Reverse Beta Oxidation','Thioesterase (ΔfadAI)','PhaG PhaJ ΔfadDEK','PhaG PhaJ tdTER BktB ΔfadBDK'])


######################################################
##### Reverse Beta-Oxidation Pathway Simulations #####
######################################################

# Load the oleochemical modified iML1515 model 
OleoiML1515 = cobra.io.read_sbml_model('Oleo_iML1515.xml')        
OleoiML1515.solver = 'glpk'

# Set the carbon source to glycerol, and remove the ATP maintenance energy 
OleoiML1515.reactions.EX_glc__D_e.lower_bound = 0
OleoiML1515.reactions.EX_glyc_e.lower_bound = -1
OleoiML1515.reactions.ATPM.lower_bound = 0

# Evaluate the theoretical yield of each product of interest. Store solutions as Reverse Beta Oxidation.
for o in objectives: 
    OleoiML1515.objective = OleoiML1515.reactions.get_by_id(objectives[o])
    FBA_Solution = OleoiML1515.optimize()
    theoreticalYield = FBA_Solution.objective_value / - FBA_Solution.fluxes['EX_glyc_e']
    iML1515_solution_df.loc[o]['Reverse Beta Oxidation'] = FBA_Solution
    iML1515_theoretical_yeilds_df.loc[o]['Reverse Beta Oxidation'] = theoreticalYield
    
    
############################################   
##### Thioesterase Pathway Simulations #####
############################################

# Reload the oleochemical modified iML1515 model 
OleoiML1515 = cobra.io.read_sbml_model('Oleo_iML1515.xml')           
OleoiML1515.solver = 'glpk'

# Set the carbon source to glycerol, and remove the ATP maintenance energy 
OleoiML1515.reactions.EX_glc__D_e.lower_bound = 0
OleoiML1515.reactions.EX_glyc_e.lower_bound = -1
OleoiML1515.reactions.ATPM.lower_bound = 0    

# Simulate fadA / fadI knockout - Prevent Reverse Beta Oxidation
OleoiML1515.genes.b2342.knock_out()
OleoiML1515.genes.b3845.knock_out()

# Heterologous PhaG Reaction Removal - Prevent PhaG reactions
OleoiML1515.reactions.PhaG6.delete()
OleoiML1515.reactions.PhaG8.delete()
OleoiML1515.reactions.PhaG10.delete()
OleoiML1515.reactions.PhaG12.delete()
OleoiML1515.reactions.PhaG14.delete()
OleoiML1515.reactions.PhaG16.delete()
OleoiML1515.reactions.CTECOAI6.delete()
OleoiML1515.reactions.HXCT.delete()

# Evaluate the theoretical yield of each product of interest.  Store solutions as Thioesterase (ΔfadAI).
for o in objectives:
    OleoiML1515.objective = OleoiML1515.reactions.get_by_id(objectives[o])
    FBA_Solution = OleoiML1515.optimize()
    theoreticalYield = FBA_Solution.objective_value / - FBA_Solution.fluxes['EX_glyc_e']
    iML1515_solution_df.loc[o]['Thioesterase (ΔfadAI)'] = FBA_Solution
    iML1515_theoretical_yeilds_df.loc[o]['Thioesterase (ΔfadAI)'] = theoreticalYield
     

###########################################   
##### PhaG -> TER Pathway Simulations ##### 
###########################################

# Reload the oleochemical modified iML1515 model 
OleoiML1515 = cobra.io.read_sbml_model('Oleo_iML1515.xml')           
OleoiML1515.solver = 'glpk'

# Set the carbon source to glycerol, and remove the ATP maintenance energy 
OleoiML1515.reactions.EX_glc__D_e.lower_bound = 0
OleoiML1515.reactions.EX_glyc_e.lower_bound = -1
OleoiML1515.reactions.ATPM.lower_bound = 0

# Prevent Reverse Beta Oxidation cycle, but allow for single elongation step via PhaG->FadB->TER->fadA
OleoiML1515.reactions.HACD1.upper_bound = 0
OleoiML1515.reactions.HACD2.upper_bound = 0
OleoiML1515.reactions.HACD3.upper_bound = 0
OleoiML1515.reactions.HACD4.upper_bound = 0
OleoiML1515.reactions.HACD5.upper_bound = 0
OleoiML1515.reactions.HACD6.upper_bound = 0
OleoiML1515.reactions.HACD7.upper_bound = 0
OleoiML1515.reactions.HACD8.upper_bound = 0

# fadD / fadK knockout 
OleoiML1515.genes.b1805.knock_out()
OleoiML1515.genes.b1701.knock_out()

# Preventing Thioesterase Route for Free Fatty Acid Synthesis
OleoiML1515.reactions.TE_C060_ACP.upper_bound = 0
OleoiML1515.reactions.TE_C080_ACP.upper_bound = 0 
OleoiML1515.reactions.TE_C100_ACP.upper_bound = 0
OleoiML1515.reactions.TE_C120_ACP.upper_bound = 0
OleoiML1515.reactions.TE_C140_ACP.upper_bound = 0
OleoiML1515.reactions.TE_C160_ACP.upper_bound = 0

#OleoiML1515.reactions.FA60ACPHi.upper_bound = 0 # Not in Model
OleoiML1515.reactions.FA80ACPHi.upper_bound = 0
OleoiML1515.reactions.FA100ACPHi.upper_bound = 0
OleoiML1515.reactions.FA120ACPHi.upper_bound = 0
OleoiML1515.reactions.FA140ACPHi.upper_bound = 0
OleoiML1515.reactions.FA160ACPHi.upper_bound = 0

# Prevent Acyl-(acyl carrier protein):phosphate acyltransferase for Free Fatty Acid Synthesis
OleoiML1515.reactions.ACPPAT120.upper_bound = 0
OleoiML1515.reactions.ACPPAT140.upper_bound = 0


# Evaluate the theoretical yield of each product of interest.  Store solutions as PhaG PhaJ tdTER BktB ΔfadBDK.
for o in objectives:
    OleoiML1515.objective = OleoiML1515.reactions.get_by_id(objectives[o])
    FBA_Solution = OleoiML1515.optimize()
    theoreticalYield = FBA_Solution.objective_value / - FBA_Solution.fluxes['EX_glyc_e']
    iML1515_solution_df.loc[o]['PhaG PhaJ tdTER BktB ΔfadBDK'] = FBA_Solution
    iML1515_theoretical_yeilds_df.loc[o]['PhaG PhaJ tdTER BktB ΔfadBDK'] = theoreticalYield
    
#########################################
##### PhaG -> FadB / FadA solutions #####
#########################################

# Reload the oleochemical modified iML1515 model 
OleoiML1515 = cobra.io.load_json_model('Oleo_iML1515.json')        
OleoiML1515.solver = 'glpk'

# Set the carbon source to glycerol, and remove the ATP maintenance energy 
OleoiML1515.reactions.EX_glc__D_e.lower_bound = 0
OleoiML1515.reactions.EX_glyc_e.lower_bound = -1
OleoiML1515.reactions.ATPM.lower_bound = 0

# fadD / fadK knockouts 
OleoiML1515.genes.b1805.knock_out()
OleoiML1515.genes.b1701.knock_out()

# fadE and thioesterase knockouts
OleoiML1515.genes.b0221.knock_out()
OleoiML1515.reactions.TER04.delete()
OleoiML1515.reactions.TER06.delete()
OleoiML1515.reactions.TER08.delete() 
OleoiML1515.reactions.TER10.delete()
OleoiML1515.reactions.TER12.delete()
OleoiML1515.reactions.TER14.delete()

# Preventing Thioesterase Route for Free Fatty Acid Synthesis
OleoiML1515.reactions.TE_C060_ACP.upper_bound = 0
OleoiML1515.reactions.TE_C080_ACP.upper_bound = 0 
OleoiML1515.reactions.TE_C100_ACP.upper_bound = 0
OleoiML1515.reactions.TE_C120_ACP.upper_bound = 0
OleoiML1515.reactions.TE_C140_ACP.upper_bound = 0
OleoiML1515.reactions.TE_C160_ACP.upper_bound = 0

#OleoiML1515.reactions.FA60ACPHi.upper_bound = 0 # Not in Model
OleoiML1515.reactions.FA80ACPHi.upper_bound = 0
OleoiML1515.reactions.FA100ACPHi.upper_bound = 0
OleoiML1515.reactions.FA120ACPHi.upper_bound = 0
OleoiML1515.reactions.FA140ACPHi.upper_bound = 0
OleoiML1515.reactions.FA160ACPHi.upper_bound = 0

# Prevent Acyl-(acyl carrier protein):phosphate acyltransferase for Free Fatty Acid Synthesis
OleoiML1515.reactions.ACPPAT120.upper_bound = 0
OleoiML1515.reactions.ACPPAT140.upper_bound = 0

# Evaluate the theoretical yield of each product of interest.  Store solutions as PhaG PhaJ ΔfadDEK.
for o in objectives:
    OleoiML1515.objective = OleoiML1515.reactions.get_by_id(objectives[o])
    FBA_Solution = OleoiML1515.optimize()
    theoreticalYield = FBA_Solution.objective_value / - FBA_Solution.fluxes['EX_glyc_e']
    iML1515_solution_df.loc[o]['PhaG PhaJ ΔfadDEK'] = FBA_Solution
    iML1515_theoretical_yeilds_df.loc[o]['PhaG PhaJ ΔfadDEK'] = theoreticalYield
    

# Display the theoretical yield results    
iML1515_theoretical_yeilds_df

,Reverse Beta Oxidation,Thioesterase (ΔfadAI),PhaG PhaJ ΔfadDEK,PhaG PhaJ tdTER BktB ΔfadBDK
Octanol,0.291667,0.267797,0.270548,0.276224
Decanol,0.233333,0.215259,0.217033,0.22067
Dodecanol,0.194444,0.179954,0.181193,0.183721
Tetradecanol,0.166667,0.154599,0.155512,0.157371
2-Heptanone,0.334746,0.30038,0.311024,0.318548
2-Nonanone,0.261589,0.235821,0.242331,0.246875
2-Undecanone,0.214674,0.194103,0.198492,0.201531
2-Tridecanone,0.182028,0.164927,0.168085,0.170259
Octanoic Acid,0.314741,0.29368,0.287273,0.29368
Decanoic Acid,0.249097,0.231672,0.227666,0.231672
